import torch_geometric

torch_geometric.__version__ >>  '1.5.0'

torch.__version__ >> '1.5.0'

torch.version.cuda >> '10.2'

torch.cuda.get_device_name(0) >> 'TITAN RTX'

## load data

In [1]:
from torch_geometric.datasets import Planetoid
import os

In [2]:
path = %pwd

In [3]:
Cora = Planetoid(root=path, name='Cora')
cora = Cora[0]

In [ ]:
C_S = Planetoid(root=path, name='CiteSeer')
cite_seer = C_S[0]

In [ ]:
PM = Planetoid(root=path, name='PubMed')
pub_med = PM[0]

In [ ]:
air = torch.load( os.path.join(path,'Air_USA.pt') )

In [ ]:
db = torch.load( os.path.join(path,'dblp.pt') )

In [ ]:
d_cora = torch.load( os.path.join(path,'dgl_cora.pt'))

In [4]:
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.utils import remove_self_loops, add_self_loops,softmax
from torch_geometric.nn.conv import MessagePassing
import numpy as np
import networkx as nx
import time

from utils import EarlyStopping

## RWN

### Aggregator 

In [5]:
# L2 normalized weights
def Rand_aw(G, n=1):
    #np.random.seed(1024)
    A = nx.to_numpy_array(G, nodelist=range(len(G))) 
    w = np.random.random(A.shape)  #uniform
    #w = np.random.randn(*A.shape) #normal  
    A = A*w + n*np.eye(A.shape[0])
    row_sum = np.sum(A**2, axis=1,keepdims=True)
    A = A/(np.sqrt(row_sum)+1e-12)
    return A

## Build a graph 

In [6]:
data = cora
data.to('cpu')
edge_index, _ = remove_self_loops(data.edge_index)
edge_index, _ = add_self_loops(edge_index, num_nodes=data.num_nodes)

edges = edge_index.numpy().T
# create a graph and feed it to Rand_aw later
rg = nx.Graph()

rg.add_edges_from(edges)

## Algorithm

In [7]:
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [8]:
class rw_GNN(MessagePassing):
    def __init__(self, in_channels, out_channels, rw, k=3, cached=True, bias=True,
                 **kwargs):
                 
        super(rw_GNN, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.cached = cached
        self.lin = Linear(in_channels, out_channels, bias=bias)

        if not torch.is_tensor(rw):
                rw = torch.tensor(rw, dtype=data.x.dtype)
        self.rw = rw.cuda()

        self.reset_parameters()

        
    def reset_parameters(self):
        self.lin.reset_parameters()
        self.cached_result = None
        self.cached_num_edges = None

    def forward(self, x, edge_index):
        """"""

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)

            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

            norm = self.rw[edge_index[1], edge_index[0]]
            
            for k in range(self.k):
                x = self.propagate(edge_index, x=x, norm=norm) 
            self.cached_result = x

        if self.cached:
            x = self.lin(self.cached_result)
        return x
    
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}({}, {}, K={})'.format(self.__class__.__name__,
                                         self.in_channels, self.out_channels,
                                         self.k)

In [9]:
aw = Rand_aw(rg, n=1)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = rw_GNN(
            num_features, num_classes, rw=aw , k=2, cached=True)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [11]:
das = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora_pytorch
  optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-1)
    
  #pubmed
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-3)
    
  #cite_seer,
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-1)
    
  #air
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-6)
    
  #dblp
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
    
  #dgl cora 
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.7, weight_decay=5e-4)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  #print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  das.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Test Accuracy 0.7970
Test Accuracy 0.7980
Test Accuracy 0.8050
Test Accuracy 0.8030
Test Accuracy 0.8150
Test Accuracy 0.8070
Test Accuracy 0.8000
Test Accuracy 0.7990
Test Accuracy 0.7980
Test Accuracy 0.7970


In [12]:
np.mean(das)

0.8019000000000001

In [13]:
np.std(das)*100

0.5467174773134637